In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)

In [2]:
# importing the data
df = pd.read_csv("brfssdata2021.csv")
df.shape


(438693, 303)

In [3]:
# dropping the data with missing dependent variable values and the people who reported mental health as none, refused and dont know
df2 = df
df2.dropna(subset=['number of days mental health not good'], inplace=True)
df2 = df2[(df2["number of days mental health not good"] != 99) & (df2["number of days mental health not good"] != 88) &
          (df2["number of days mental health not good"] != 77) & (df2["number of days mental health not good"] != 30) &
          (df2["binge drinking calculated variable"] != 9)]


df2 = df2[(df2['do you now use e-cigarettes, every day,'] != 7) & (df2['do you now use e-cigarettes, every day,'] != 9) &
          (df2['frequency of days now smoking'] != 7) & (
              df2['frequency of days now smoking'] != 9) & (df2["smoked at least 100 cigarettes"] != 7)
          & (df2["smoked at least 100 cigarettes"] != 9)]
df2["Smoking"] = np.where((df2["smoked at least 100 cigarettes"] == 1) | (df2["frequency of days now smoking"] == 1) | (df2["frequency of days now smoking"] == 2), "cigarettes", 
np.where((df2["do you now use e-cigarettes, every day,"] == 1) | (df2["do you now use e-cigarettes, every day,"] == 2), "e-cigarettes", 
np.where((df2["smoked at least 100 cigarettes"] == 2) | (df2["frequency of days now smoking"] == 3) | (df2["do you now use e-cigarettes, every day,"] == 3) | (df2["do you now use e-cigarettes, every day,"] == 4), "no", "none")))


df2.rename({'computed body mass index categories': 'BMI', 'computed height in inches': 'Height_Inch', 
            "computed weight in kilograms": "Weight_kg", "binge drinking calculated variable": "binge_drinking", 
            "imputed age in six groups": "Age_in_Groups"}, axis=1, inplace=True)

df2.shape


(125118, 304)

In [4]:
# dropping columns with more than 70% missing data
df3 = df2.dropna(axis='columns', thresh = int(0.70 * len(df2)))
df3.shape

(125118, 150)

In [5]:
# storing the calculated, computed, missing variables in an array
calculated_cols = [i for i in df3.columns if 'calculated' in i.lower() or 'computed' in i.lower()
                  or 'missing' in i.lower() or 'interview' in i.lower() ]
print(len(calculated_cols))
calculated_cols

45


['interview date',
 'interview month',
 'interview day',
 'interview year',
 'child hispanic, latino/a, or spanish origin calculated variable',
 'computed physical health status',
 'computed mental health status',
 'leisure time physical activity calculated variable',
 'high blood pressure calculated variable',
 'cholesterol checked calculated variable',
 'high cholesterol calculated variable',
 'lifetime asthma calculated variable',
 'current asthma calculated variable',
 'computed asthma status',
 'computed preferred race',
 'calculated non-hispanic race including multiracial',
 'hispanic, latino/a, or spanish origin calculated variable',
 'computed race-ethnicity grouping',
 'computed non-hispanic whites/all others',
 'computed five level race/ethnicity categories',
 'computed race groups used for internet prevalence tables',
 'calculated sex variable',
 'reported age in two age groups calculated variable',
 'computed height in meters',
 'computed body mass index',
 'overweight or o

In [6]:
# droppind the calculated variables
df4 = df3.drop(calculated_cols, axis=1)
df4.shape

(125118, 105)

In [7]:
# dropping all the missing data from the dataset
df4.dropna(inplace=True)
df4.shape

(69172, 105)

In [8]:
# reseting the index
df5 = df4.reset_index(drop=True)
df5.shape

(69172, 105)

In [9]:
# mapping the state code with state names
# reading the state names with the corresponding column names from the csv file
df_state_mapping = pd.read_csv("state mapping.csv")
di_state = df_state_mapping.set_index("id")["state"].to_dict()

# converting the month number to month name
df5["file month"] = df5["file month"].apply(str)

# mapping the state name with state number
df5["state fips code"] = df5["state fips code"].map(di_state)
df5.head(2)

,state fips code,file month,final disposition,annual sequence number,primary sampling unit,safe time to talk?,correct phone number?,is this a cell phone?,are you 18 years of age or older?.1,are you male or female?.2,...,french fry intake in times per day,total fruits consumed per day,total vegetables consumed per day,consume fruit 1 or more times per day,consume vegetables 1 or more times per day,reported consuming fruit >16/day,reported consuming vegetables >23/day,fruit exclusion from analyses,vegetable exclusion from analyses,Smoking
0,Alabama,1,1100,2021001136,2021001136,1.0,1.0,1.0,1.0,2.0,...,10.0,43.0,162.0,2,1,1,1,0,0,no
1,Alabama,1,1100,2021001140,2021001140,1.0,1.0,1.0,1.0,2.0,...,0.0,313.0,228.0,1,1,1,1,0,0,no


In [10]:
df5.shape


(69172, 105)

In [11]:
# removing the data where the reported response is "Dont know" or "refused" for various columns
df5 = df5[(df5['currently taking medicine for high cholesterol'] != 7) & (df5['currently taking medicine for high cholesterol'] != 9) &
          (df5['ever diagnosed with heart attack'] != 7) & (df5['ever diagnosed with heart attack'] != 9) &
          (df5['ever told had asthma'] != 7) & (df5['ever told had asthma'] != 9) &
          (df5['ever told you had c.o.p.d. emphysema or chronic bronchitis?'] != 7) & (df5['ever told you had c.o.p.d. emphysema or chronic bronchitis?'] != 9) &
          (df5['(ever told) you had skin cancer?'] != 7) & (df5['(ever told) you had skin cancer?'] != 9) &
          (df5['(ever told) you had any other types of cancer'] != 7) & (df5['(ever told) you had any other types of cancer'] != 9) &
          (df5['ever told you have kidney disease?'] != 7) & (df5['ever told you have kidney disease?'] != 9) &
          (df5['(ever told) you had diabetes'] != 7) & (df5['(ever told) you had diabetes'] != 9) &
          (df5['told have arthritis'] != 7) & (df5['told have arthritis'] != 9) & 
          (df5['marital status'] != 9) & 
          (df5['education level'] != 9) &
          (df5['own or rent home'] != 7) & (df5['own or rent home'] != 9) & 
          (df5['what is primary source of health insurance'] != 77) & (df5['what is primary source of health insurance'] != 99) & 
          (df5["general health"] != 7) & (df5["general health"] != 9) &
          (df5['have personal health care provider?'] != 7) & (df5['have personal health care provider?'] != 9) &
          (df5['could not afford to see doctor'] != 7) & (df5['could not afford to see doctor'] != 9) & 
          (df5['length of time since last routine checkup '] != 7) & (df5['length of time since last routine checkup '] != 9) & 
          (df5['exercise in past 30 days'] != 7) & (df5['exercise in past 30 days'] != 9) &
          (df5['are you a veteran'] != 7) & (df5['are you a veteran'] != 9) & (df5['employment status'] != 9) & (df5['number of children in household'] != 99) & 
          (df5['income level'] != 99) & (df5['income level'] != 99) &
          (df5['difficulty walking or climbing stairs'] != 7) & (df5['difficulty walking or climbing stairs'] != 9) &
          (df5['difficulty dressing or bathing'] != 7) & (df5['difficulty dressing or bathing'] != 9) &
          (df5['difficulty doing errands alone'] != 7) & (df5['difficulty doing errands alone'] != 9) &
          (df5['are you deaf or do you have serious difficulty hearing?'] != 7) & (df5['are you deaf or do you have serious difficulty hearing?'] != 9) &
          (df5['blind or difficulty seeing'] != 7) & (df5['blind or difficulty seeing'] != 9) &
          (df5['difficulty concentrating or remembering'] != 7) & (df5['difficulty concentrating or remembering'] != 9) &
          (df5['adult flu shot/spray past 12 mos'] != 7) & (df5['adult flu shot/spray past 12 mos'] != 9) &
          (df5['ever tested h.i.v.'] != 7) & (df5['ever tested h.i.v.'] != 9) &
          (df5['how many times did you eat fruit?'] != 777) & (df5['how many times did you eat fruit?'] != 999) & 
          (df5['how many times did you eat dark green vegetables?'] != 777) & (df5['how many times did you eat dark green vegetables?'] != 999) &
          (df5['how often do you eat french fries or fried potatoes'] != 777) & (df5['how often do you eat french fries or fried potatoes'] != 999) &
          (df5['how many times did you eat other vegetables?'] != 777) & (df5['how many times did you eat other vegetables?'] != 999) &
          (df5["number of days physical health not good"] != 77) & (df5["number of days physical health not good"] != 99)]


df5.shape


(57999, 105)

In [12]:
# Feature Engineering

df5["bad_physical_health_days"] = np.where((df5["number of days physical health not good"] == 88), 0, df5["number of days physical health not good"])

df5['sex of respondent'] = np.where(df5['sex of respondent'] == 1, 'male', 'female')

df5['Health_Insurance'] = df5['what is primary source of health insurance'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
df5['Health_Insurance'] = np.where(df5['Health_Insurance']== True, "yes", "no")

df5['general_health'] = np.where((df5['general health'] == 1) | (df5['general health'] == 2) | (df5['general health'] == 3), 'good', "poor")

df5['Personal_Health_Care_Provider'] = np.where((df5['have personal health care provider?'] == 1) | 
                                                (df5['have personal health care provider?'] == 2), 'Yes', "No")
df5['Afford_to_see_doctor']= np.where((df5['could not afford to see doctor']==1),'No','Yes')

df5['routine_checkup ']= np.where(df5['length of time since last routine checkup ']==1,'Within past year',
                                                   np.where(df5['length of time since last routine checkup ']==2,'Within past 2 years',
                                                   np.where(df5['length of time since last routine checkup ']==3,'Within past 5 years',
                                                   np.where(df5['length of time since last routine checkup ']==4,'5 or more years ago','Never'))))

df5['exercise in past 30 days']= np.where(df5['exercise in past 30 days']==1,'Yes','No')

df5['currently taking medicine for high cholesterol']= np.where(df5['currently taking medicine for high cholesterol']==1,'Yes','No')

df5['Heart_Diseases'] = np.where((df5['ever diagnosed with heart attack'] == 1) | (df5['ever diagnosed with angina or coronary heart disesase'] == 1) |
                                (df5['ever diagnosed with a stroke'] == 1), 'Yes', 'No')

df5['ever told had asthma'] = np.where((df5['ever told had asthma'] == 1),'Yes','No')

df5['ever told you had c.o.p.d. emphysema or chronic bronchitis?'] = np.where((df5['ever told you had c.o.p.d. emphysema or chronic bronchitis?'] == 1),'Yes','No')

df5['Cancer'] = np.where((df5['(ever told) you had skin cancer?'] == 1) | (df5['(ever told) you had any other types of cancer'] == 1),'Yes','No')

df5['ever told you have kidney disease?'] = np.where((df5['ever told you have kidney disease?'] == 1),'Yes','No')

df5['(ever told) you had diabetes'] = np.where((df5['(ever told) you had diabetes'] == 1)|(df5['(ever told) you had diabetes'] == 2),'Yes','No')

df5['told have arthritis'] = np.where((df5['told have arthritis'] == 1),'Yes','No')

df5['marital_status']= np.where(df5['marital status']==1,'Married',
                       np.where((df5['marital status']==2)|(df5['marital status']==4),'Separated',
                       np.where(df5['marital status']==3,'Widowed','Unmarried')))

df5['education_level']= np.where(df5['education level']==1,'Kindergarten school',
                        np.where(df5['education level']==2,'Elementary School',
                        np.where((df5['education level']==3) |(df5['education level']==4) ,'High School','College')))

df5['own or rent home'] = np.where(df5['own or rent home']==1,'Own',
                        np.where(df5['own or rent home']==2,'Rent','Other'))

df5['are you a veteran'] = np.where(df5['are you a veteran']==1,'yes', 'no')

df5['employment_status']= np.where((df5['employment status']==1) | (df5['employment status']==2),'Employed',
                                    np.where((df5['employment status']==3) | (df5['employment status']==4) | (df5['employment status']==8),'Unemployed',
                                    np.where(df5['employment status']==5,'Homemaker',
                                    np.where(df5['employment status']==6,'Student', "Retired"))))

df5['number of children in household']= np.where((df5['number of children in household']==88) ,'None',
                                            np.where((df5['number of children in household']==1) ,'1',
                                            np.where(df5['number of children in household']==2,'2',
                                            np.where(df5['number of children in household']==3,'3', "More than 3"))))     

df5['Income_Level']= np.where((df5['income level'].isin([1,2,3,4])) ,'less than 25k',
                                            np.where((df5['income level'].isin([5,6])) ,'25k-50k',
                                            np.where(df5['income level'].isin([7,8]),'50k-100k',
                                            np.where(df5['income level'].isin([9,10]),'100k-200k', "More than 200k"))))    

df5['Difficulty_Doing_Errands']= np.where((df5['difficulty walking or climbing stairs']==1) | 
                                            (df5['difficulty dressing or bathing']==1)  | (df5['difficulty doing errands alone']==1),'Yes',"No")  

df5['Difficulty_Hearing']= np.where((df5['are you deaf or do you have serious difficulty hearing?']==1) ,'Yes',"No")

df5['Difficulty_Seeing']= np.where((df5['blind or difficulty seeing']==1) ,'Yes',"No")

df5['Difficulty_Concentrating']= np.where((df5['difficulty concentrating or remembering']==1) ,'Yes',"No")   

df5['Flushot']= np.where((df5['adult flu shot/spray past 12 mos']==1) ,'Yes', "No")

df5['County_Type']= np.where((df5['urban/rural status']==1) ,'Urban', "Rural")

df5["Race"] = np.where(df5["imputed race/ethnicity value"]== 1, "White",
                np.where(df5["imputed race/ethnicity value"]==2, "Black",
                np.where(df5["imputed race/ethnicity value"]==3, "Asian",
                np.where(df5["imputed race/ethnicity value"]==4, "American Indian",
                np.where(df5["imputed race/ethnicity value"]==5, "Hispanic", "Other")))))

df5["Age"] = np.where(df5["Age_in_Groups"]== 1, "18-24",
                        np.where(df5["Age_in_Groups"]==2, "25-34",
                        np.where(df5["Age_in_Groups"]==3, "35-44",
                        np.where(df5["Age_in_Groups"]==4, "45-54",
                        np.where(df5["Age_in_Groups"]==5, "55-64", "64+")))))

df5["Binge_Drinking"] = np.where(df5["binge_drinking"]==1, "no", "yes")

df5['Diet']= np.where((df5['how many times did you eat fruit?']>100) & (df5['how many times did you eat fruit?']<300)&
                      (df5['how many times did you eat dark green vegetables?']>100) & (df5['how many times did you eat dark green vegetables?']<300) &
                      (df5['how many times did you eat other vegetables?']>100) & (df5['how many times did you eat other vegetables?']<300) &
                      (df5['how often do you eat french fries or fried potatoes']==555), 'healthy diet','mixed diet')

df5["BMI"] = np.where(df5["BMI"]== 1, "Underweight",
                np.where(df5["BMI"]==2, "NormalWeight",
                np.where(df5["BMI"]==3, "OverWeight","Obese")))

df5["Weight_kg"] = df5["Weight_kg"]/100

df5.shape

(57999, 127)

In [13]:
# renaming the column names
df5.rename({'currently taking medicine for high cholesterol': 'Cholesterol', 'ever told had asthma': 'Asthma', 
            'ever told you had c.o.p.d. emphysema or chronic bronchitis?': 'Chronic_Bronchitis',
            'ever told you have kidney disease?': 'Kidney_Disease', '(ever told) you had diabetes': 'Diabetes', 
            'told have arthritis': 'Arthritis', 'own or rent home': 'House', 'sex of respondent': 'Sex', 
            'exercise in past 30 days': 'Exercise', 'are you a veteran': 'Veteran', 
            "number of children in household" :"Children", "state fips code" : "State", "number of days mental health not good" : "bad_mental_health_days"}, axis=1, inplace=True)

df5.shape


(57999, 127)

In [14]:
# dropping the columns
manual_columns = [	"final disposition",	"annual sequence number",	"primary sampling unit",	"safe time to talk?",	"correct phone number?",	
                    "is this a cell phone?", "are you male or female?.2",	"do you live in a private residence?",	"do you currently live in  ____(state)___",
                    "do you also have a landline telephone?",	"number of adults in household.1", "poor physical or mental health",  
                    "how long since cholesterol checked", "do you have a cell phone for personal us", "questionnaire version identifier",
                    "language identifier", "sample design stratification variable", "stratum weight", "raw weighting factor used in raking",
                    "design weight used in raking", "dual phone use categories", "truncated design weight used in adult combined land line and cell phone raking",
                    "final weight: land-line and cell-phone data","ever told blood pressure high", "(ever told) you had a depressive disorder",
                    "(ever told) you had skin cancer?","(ever told) you had any other types of cancer",
                    "respondents that have ever reported having coronary heart disease (chd) or myocardial infarction (mi)", 
                    "respondents diagnosed with arthritis", "limited usual activities", "limited work activities", 
                    "reported age in five-year age categories", "ever told cholesterol is high","imputed age value collapsed above 80", 
                    "drink any alcoholic beverages in past 30 days", "french fry intake in times per day", 
                    "total fruits consumed per day", "total vegetables consumed per day", "consume fruit 1 or more times per day", 
                    "consume vegetables 1 or more times per day", "reported consuming fruit >16/day", "reported consuming vegetables >23/day", 
                    "fruit exclusion from analyses", "vegetable exclusion from analyses", 'have personal health care provider?', 
                    'what is primary source of health insurance', 'could not afford to see doctor', 'ever diagnosed with heart attack', 
                    'ever diagnosed with angina or coronary heart disesase', 'ever diagnosed with a stroke', 'difficulty walking or climbing stairs',
                    'difficulty dressing or bathing', 'difficulty doing errands alone', 'are you deaf or do you have serious difficulty hearing?',
                    'blind or difficulty seeing', 'difficulty concentrating or remembering', "smoked at least 100 cigarettes", 
                    "do you now use e-cigarettes, every day,", 'reported weight in pounds', 'reported height in feet and inches',
                    'use of smokeless tobacco products', "have any health insurance", "adults with good or better health", "pneumonia shot ever",
                    "adult flu shot/spray past 12 mos", "urban/rural status", "respondents aged 18-64 with health insurance", "imputed race/ethnicity value",
                    "ever tested h.i.v.", "days in past 30 had alcoholic beverage", "metropolitan status",'how many times did you eat fruit?', 
                    'how many times did you drink 100 percent pure fruit juices?','how often do you eat potatoes', 
                    'how many times did you eat dark green vegetables?', 'how many times did you eat other vegetables?', 
                    'how often do you eat french fries or fried potatoes', 'file month', "are you 18 years of age or older?.1",
                    "binge_drinking", "number of days physical health not good", "income level", "Age_in_Groups",
                    "length of time since last routine checkup ", "marital status", "education level", "employment status", "general health"]
df5.drop(manual_columns, axis=1, inplace = True)

In [15]:
df5["general_health"]

0        good
1        good
3        poor
5        good
7        poor
         ... 
69166    good
69167    good
69168    good
69169    good
69171    good
Name: general_health, Length: 57999, dtype: object

In [16]:
df5.columns

Index(['State', 'Sex', 'bad_mental_health_days', 'Exercise', 'Cholesterol',
       'Asthma', 'Chronic_Bronchitis', 'Kidney_Disease', 'Diabetes',
       'Arthritis', 'House', 'Veteran', 'Children', 'Height_Inch', 'Weight_kg',
       'BMI', 'Smoking', 'bad_physical_health_days', 'Health_Insurance',
       'general_health', 'Personal_Health_Care_Provider',
       'Afford_to_see_doctor', 'routine_checkup ', 'Heart_Diseases', 'Cancer',
       'marital_status', 'education_level', 'employment_status',
       'Income_Level', 'Difficulty_Doing_Errands', 'Difficulty_Hearing',
       'Difficulty_Seeing', 'Difficulty_Concentrating', 'Flushot',
       'County_Type', 'Race', 'Age', 'Binge_Drinking', 'Diet'],
      dtype='object')

In [17]:
df5.shape

(57999, 39)

In [18]:
# # merging unemployment rate and poverty rate in each state
# df_unemp = pd.read_excel("unemploymentrate_2021.xlsx")
# df_poverty = pd.read_csv("poverty_rate_2021.csv")
# df_unemp.rename({'rate': 'unemployment_rate'}, axis=1, inplace=True)
# df_health_emp = pd.merge(df5, df_unemp, on='State')
# df_health_emp = pd.merge(df_health_emp, df_poverty, on='State')
# df_health_emp.shape
# df_health_emp.columns


In [19]:
df_final = df5
df_final.to_excel("df_final.xlsx", sheet_name = 'Mental Health Predictors', index=False)

In [20]:
# df5[df5["Race"] == "Other"]["bad_mental_health_days"].mean()

In [21]:
# states = df5["State"].unique()

# for i in states:
#     print(f"{i} =" + str(df5[df5['State'] == i]["bad_mental_health_days"].mean()))